In [0]:
from google.colab import drive
drive.mount('/gdrive')

# Set up a Kaggle environment to download a Kaggle dataset (Only need to do it for the first time)
# !pip uninstall kaggle
# !pip install --upgrade pip
# !pip install kaggle
# %cp /gdrive/'My Drive'/kaggle.json ~/.kaggle/

In [0]:
%cd /gdrive/'My Drive'/

In [0]:
# Download a library including LIME implementation (Only need to do it for the first time)
# !git clone https://github.com/aildnont/covid-cxr.git

In [0]:
%cd covid-cxr/

In [0]:
# Download a library including SHAP implementation (Only need to do it for the first time)
# %cd src/
# !git clone https://github.com/slundberg/shap.git
# %cd ..

In [0]:
# After executing the following instructions, one needs to restart runtime.
!pip install -r requirements.txt
!pip install shap

In [0]:
%cd data/

In [0]:
# Download COVID-19 datasets and unzip them if needed (Only need to do it for the first time)
# !git clone https://github.com/ieee8023/covid-chestxray-dataset.git
# !git clone https://github.com/agchung/Figure1-COVID-chestxray-dataset.git
# !kaggle competitions download -c rsna-pneumonia-detection-challenge
# !unzip rsna-pneumonia-detection-challenge.zip -d rsna/

In [0]:
# Create ramdisks for directories containing corresponding COVID-19 datasets
!mount -t tmpfs -o size=500m tmpfs covid-chestxray-dataset/
!mount | tail -n 1

!mount -t tmpfs -o size=50m tmpfs Figure1-COVID-chestxray-dataset/
!mount | tail -n 1

!mount -t tmpfs -o size=4450m tmpfs rsna/
!mount | tail -n 1

In [0]:
# Download COVID-19 datasets and unzip them if needed in ramdisks
%cd covid-chestxray-dataset/
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git .
%cd ..

%cd Figure1-COVID-chestxray-dataset/
!git clone https://github.com/agchung/Figure1-COVID-chestxray-dataset.git .
%cd ..

!unzip rsna-pneumonia-detection-challenge.zip -d rsna/
%cd ..

In [0]:
# Create Pandas DataFrames of filenames and labels and three corresponding files (train_set.csv, val_set.csv and test_set.csv)
!python src/data/preprocess.py

In [0]:
import os
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-10.0/extras/CUPTI/lib64/'
!echo $LD_LIBRARY_PATH

In [0]:
# Traning model resnet101v2 for binary and multi-class classifiers
!python src/train.py

In [0]:
# Applying LIME on the test set to produce binary and multi-class classifiers' results
# To switch classifiers, one needs to modify config.yml and executes "!python src/data/preprocess.py" accordingly.
!python src/lime_explain.py

In [0]:
%load_ext tensorboard

In [0]:
# Applying tensorboard to print results for training and validation
%tensorboard --logdir=results/logs/training/

In [0]:
# Kill tensorboard process
!kill $(ps | awk '{if ($4 == "tensorboard") {print $1}}')

In [0]:
# Applying SHAP on the test set to produce binary and multi-class classifiers' results
# To switch classifiers, one needs to modify config.yml and executes "!python src/data/preprocess.py" accordingly.
import multiprocessing
from src import shap_explain
p = multiprocessing.Process(target=shap_explain.main)
p.start()
p.join()